In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# MODIFY HERE TO YOUR PATH IN GOOGLE DRIVE
%cd /content/gdrive/MyDrive/CryptoML

In [ ]:
!pip install shap

In [4]:
import speck as sp
import numpy as np

def evaluate(net,X,Y):
    Z = net.predict(X,batch_size=10000).flatten()
    Zbin = (Z > 0.5)
    diff = Y - Z 
    mse = np.mean(diff*diff)
    n = len(Z) 
    n0 = np.sum(Y==0) 
    n1 = np.sum(Y==1)
    acc = np.sum(Zbin == Y) / n
    tpr = np.sum(Zbin[Y==1]) / n1
    tnr = np.sum(Zbin[Y==0] == 0) / n0
    mreal = np.median(Z[Y==1])
    high_random = np.sum(Z[Y==0] > mreal) / n0
    print("Accuracy: ", acc, "TPR: ", tpr, "TNR: ", tnr, "MSE:", mse)
    print("Percentage of random pairs with score higher than median of real pairs:", 100 * high_random)

In [7]:
import shap
from keras.models import load_model

dir = './models/'

# MODIFY HERE
num_of_plaintexts = 4
num_rounds = 5
num_of_samples = 10
net = load_model(dir + "best5depth10.h5")

if num_of_plaintexts == 2:
    X,Y = sp.make_train_data_2pt(10**6, num_rounds)
    Xr, Yr = sp.real_differences_data_2pt(10**6, num_rounds)

else:
    X,Y = sp.make_train_data_4pt(10**6, num_rounds)
    Xr, Yr = sp.real_differences_data_4pt(10**6, num_rounds)

In [ ]:
print('Testing neural distinguishers against blocks in the ordinary real vs random setting')
evaluate(net, X, Y)

shap.initjs()
explainer = shap.KernelExplainer(net.predict, X[:num_of_samples])
shap_values = explainer.shap_values(X[:num_of_samples])
shap.force_plot(explainer.expected_value[0], shap_values[0], X)

In [ ]:
print('Testing real differences setting now.')
evaluate(net, Xr, Yr)

shap.initjs()
explainer = shap.KernelExplainer(net.predict, Xr[:num_of_samples])
shap_values = explainer.shap_values(Xr[:num_of_samples])
shap.force_plot(explainer.expected_value[0], shap_values[0], Xr)